In [24]:
# Importing Libraries
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import cv2
import os
import PIL
import random
from time import time
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Conv2D, Activation, Flatten, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import datasets, layers, models

In [25]:
# Determining the data paths
train_path = "../input/sprints-ai-and-ml-competition-2022/Scenes training set/Scenes training set"
test_path = "../input/sprints-ai-and-ml-competition-2022/Scenes testing test/Scenes testing test"

In [26]:
img_width = 150 # Define the width of images 
img_height = 150 # Define the height of images 
batch_size = 32 # Define the size of the batch

In [27]:
# Loading data images and labels function for training
def load_img(pth):
    Img = []
    Lbl = []  
    lbl = 0       
    for i in os.listdir(pth): 
            print(i)
            if i == 'buildings': 
                lbl = 0
            elif i == 'forest':
                lbl = 1
            elif i == 'glacier':
                lbl = 2
            elif i == 'mountain':
                lbl = 3
            elif i == 'sea':
                lbl = 4
            elif i == 'street':
                lbl = 5
            
            for m in os.listdir(pth+'/'+i): 
               
                image = cv2.imread(pth+'/'+i+'/'+m) 
                image = cv2.resize(image,(150,150)) 
                Img.append(image)
                Lbl.append(lbl)
        
    return shuffle(Img, Lbl, random_state = 1) 

def class_lbl(index):
    labels = {0:'buildings', 1:'forest', 2:'glacier', 3:'mountain', 4:'sea',5:'street' }
    
    return labels[index]

In [28]:
# Converting images and labels into NumPy arrays
img, lbl = load_img(train_path) 
img = np.array(img) 
lbl = np.array(lbl)

In [29]:
# Plotting random images and their labels
fig ,ax = plt.subplots(3,3)
fig.subplots_adjust(0,0,3,3)
for n in range (3):
    for i in range(3):
        var = random.randint(0,len(img))
        ax[n,i].imshow(img[var])
        ax[n,i].set_title(class_lbl(lbl[var]))
    

In [40]:
# Building Keras Sequential model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.summary()

In [41]:
# Adding some layer (Tuning the model)
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='softmax'))
model.add(layers.Dense(10))

In [42]:
# Compiling the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [43]:
# Fitting the model
history = model.fit(img, lbl, epochs=20, validation_split=0.2)

In [45]:
# Reading the empty submit file
df = pd.read_csv('../input/sprints-ai-and-ml-competition-2022/submit.csv')

In [46]:
# Predicting the test set and storing the predictions in a DataFrame
from tensorflow import expand_dims
images=os.listdir(test_path)

for i,image in enumerate(images):
    arr=cv2.imread(os.path.join(test_path,image))
    arr = cv2.resize(arr, (150, 150))
    arr = expand_dims(arr, axis=0)
    name=image
    shape=arr.shape
    prediction=np.argmax(model.predict(arr))
    df.loc[i,"Image"]=str(name)
    df.loc[i,"Label"]=prediction

In [47]:
# Appending results to a CSV file
df.to_csv('./submit.csv',index = False)